In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from pygam import LinearGAM, s
from arch import arch_model
from statsmodels.tsa.stattools import acf
from scipy.signal import periodogram, find_peaks
from scipy.stats import ks_2samp, entropy
from sklearn.metrics import pairwise_distances, accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from IPython.display import display

In [5]:
# Reproducible RNG
RNG = np.random.default_rng(42)

In [6]:
# 1. Load & Preprocess
csv_path = './data/weather/temp.csv'
df = pd.read_csv(csv_path, parse_dates=['date']).set_index('date')
y = df['T (degC)'].values
n = len(y)
train_end = int(0.8 * n)
y_train, y_hold = y[:train_end], y[train_end:]
print(f"Training on first {train_end} points, holding out {n-train_end} points for evaluation.")

time = np.arange(n)

/var/folders/kw/zxdjxts97j33lc5_j5mmjgtm0000gn/T/ipykernel_8352/1464015024.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(csv_path, parse_dates=['date']).set_index('date')


Training on first 42156 points, holding out 10540 points for evaluation.


In [7]:
# 2. STL Decomposition for trend & seasonal
stl = STL(y_train, period=144, robust=True)
res = stl.fit()
trend_train = res.trend
season_train = res.seasonal
resid_train = res.resid

# Extend trend & season to full length via GAM and periodic spline
# 2.1 GAM trend fit
gam = LinearGAM(s(0)).fit(time[:train_end].reshape(-1,1), trend_train)
trend_full = gam.predict(time.reshape(-1,1))
# 2.2 Periodic spline for seasonality
# fit on one period cycle index within year
period = 144
cycle_idx = (time % period).reshape(-1,1)
spline = LinearGAM(s(0, periodic=True)).fit(cycle_idx[:train_end], season_train)
season_full = spline.predict(cycle_idx)

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# 3. Fit GARCH on residuals for heteroskedastic noise
am = arch_model(resid_train, vol='Garch', p=1, q=1, dist='normal').fit(disp='off')
# Simulate residuals + volatility
resid_sim = am.simulate(am.params, n)
sigma_sim = np.sqrt(resid_sim['sigma2'])
eps_sim = RNG.standard_normal(n) * sigma_sim
# Generate synthetic series
y_synth = trend_full + season_full + eps_sim
print("Synthetic series generated with STL/GAM trend, periodic spline season, and GARCH residuals.")

In [ ]:
# 4. Hold-out real & synthetic
y_real_hold = y_hold
y_synth_hold = y_synth[train_end:]

In [ ]:
# 5. Metric definitions with improved formulas
# 5.1 Block bootstrap real series
block_size = 144  # daily block for residuals

def block_bootstrap_real(rng):
    # Create blocks from training residuals
    blocks = [resid_train[i:i+block_size] for i in range(0, len(resid_train), block_size)]
    num_blocks = int(np.ceil(len(resid_train) / block_size))
    chosen = rng.choice(len(blocks), size=num_blocks, replace=True)
    bs = np.concatenate([blocks[i] for i in chosen])[:len(resid_train)]
    # Extend to full length
    full_bs = np.concatenate([bs, bs[:n-len(bs)]])
    return trend_full + season_full + full_bs

# 5.2 Synthetic generator

def generate_synth(rng):
    # Seed numpy for GARCH simulate reproducibility
    np.random.seed(rng.integers(0, 2**32))
    sim = am.simulate(am.params, n)
    sigma = np.sqrt(sim['sigma2'])
    eps = np.random.randn(n) * sigma
    return trend_full + season_full + eps

# 5.3 Metric functions
NLAGS = 30

def weighted_acf_l2(x, y):
    a = acf(x, nlags=NLAGS, fft=True)
    b = acf(y, nlags=NLAGS, fft=True)
    w = 1 / (np.arange(NLAGS+1) + 1)
    return np.sum(w * (a - b)**2)

def spectral_kl(x, y):
    p = periodogram(x)[1]; q = periodogram(y)[1]
    P = p / p.sum(); Q = q / q.sum()
    return entropy(P, Q)

def classifier_auc(r, s, w):
    # non-overlapping windows
    X, Y = [], []
    for i in range(0, len(r)-w, w):
        X.append(r[i:i+w]); Y.append(0)
    for i in range(0, len(s)-w, w):
        X.append(s[i:i+w]); Y.append(1)
    X = np.array(X).reshape(-1, w); Y = np.array(Y)
    Xtr, Xte, Ytr, Yte = train_test_split(X, Y, test_size=0.3, random_state=42)
    probs = RandomForestClassifier(n_estimators=100, random_state=42).fit(Xtr, Ytr).predict_proba(Xte)[:,1]
    return roc_auc_score(Yte, probs)

# 5.4 TSTR/TRTS forecasting utility
def tstr(r_hold, s):
    # train on synthetic training portion, test on real hold
    Xs, ys = [], []
    for i in range(train_end-window):
        Xs.append(s[i:i+window]); ys.append(s[i+window])
    model = Ridge().fit(np.array(Xs), np.array(ys))
    # forecast real hold
    preds = []
    for i in range(window, len(r_hold)):
        preds.append(model.predict(r_hold[i-window:i].reshape(1,-1))[0])
    return np.mean((r_hold[window:] - np.array(preds))**2)

def trts(r, s_hold):
    # train on real training, test on synthetic hold
    Xr, yr = [], []
    for i in range(train_end-window):
        Xr.append(r[i:i+window]); yr.append(r[i+window])
    model = Ridge().fit(np.array(Xr), np.array(yr))
    preds = []
    for i in range(window, len(s_hold)):
        preds.append(model.predict(s_hold[i-window:i].reshape(1,-1))[0])
    return np.mean((s_hold[window:] - np.array(preds))**2)


In [ ]:
# 6. Bootstrap all metrics
def bootstrap_pair(metric_fn, B=200):
    vals = []
    for seed in range(B):
        rng = np.random.default_rng(seed)
        yb = block_bootstrap_real(rng)[train_end:]
        ys = generate_synth(rng)[train_end:]
        vals.append(metric_fn(yb, ys))
    arr = np.array(vals)
    return arr.mean(), np.percentile(arr, [2.5, 97.5])

In [ ]:
# 7. Compute metrics on hold-out with CIs
window = 144

# Weighted ACF
wac_mean, wac_ci = bootstrap_pair(weighted_acf_l2)
print(f"Weighted ACF L2: {wac_mean:.4e}, CI={wac_ci}")
# Spectral KL
skl_mean, skl_ci = bootstrap_pair(spectral_kl)
print(f"Spectral KL: {skl_mean:.4e}, CI={skl_ci}")
# Classifier AUC
auc_mean, auc_ci = bootstrap_pair(lambda a,b: classifier_auc(a,b,window))
print(f"Classifier AUC: {auc_mean:.4f}, CI={auc_ci}")
# TSTR/TRTS (no bootstrap)
print(f"TSTR MSE: {tstr(y_real_hold, y_synth):.4f}")
print(f"TRTS MSE: {trts(y_train, y_synth_hold):.4f}")

In [ ]:
# 8. Summary
print("
Bootstrap metrics summary (hold-out):")
display(pd.DataFrame({
    'Metric': ['Weighted ACF L2','Spectral KL','Classifier AUC'],
    'Mean': [wac_mean, skl_mean, auc_mean],
    'CI Lower': [wac_ci[0], skl_ci[0], auc_ci[0]],
    'CI Upper': [wac_ci[1], skl_ci[1], auc_ci[1]]
}))